# MNIST 손글씨 데이터 세트 불러오기

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets

In [ ]:
mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

**Fully Connected Layer**를 사용하기 위해 평탄화, 필요에 따라 OHE을 진행....

# Keras란?
* `tensorflow`를 기반으로 하는 유명한 논문, 레이어들을 구현을 해 놓은 패키지

# Fully Connected Layer 모델링 하기
* 레이어들은 `tf.keras.layers` 패키지에 모두 들어 있다.
* `Flatten Layer` : 배치(BATCH)를 제외한 평탄화 담당
  * `numpy` 할 때는 `reshape`, 배치 개수 따로 빼고 해야 했는데, 여기선 그럴 필요가 없다.
* `Dense Layer` : `Affine`연산(신경망의 행렬 내적 계산). **Fully Connected**
  * `Dense Layer` 파라미터에 `activation`을 지정 가능
* `Activation Layer` : 활성화 함수 레이어(선택에 따라서 사용을 안할 수도 있다.)

In [ ]:
X_train = X_train / 255.0
X_test  = X_test / 255.0

# Fully Connected Layer 모델링
* 계산그래프를 생성했다고 보면 됩니다.
* `Model`객체 생성시에 계산그래프 생성

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Activation, Input

input = Input(shape=(28, 28)) # 배치를 제외한 shape을 지정 (순수한 데이터만의 shape)
net = Flatten()(input) # 평탄화

# 은닉층
net = Dense(512)(net)
net = Activation("relu")(net)

net = Dense(256)(net)
net = Activation("relu")(net)

# 출력층
net = Dense(10)(net)
net = Activation("softmax")(net)

# 모델 생성 - 계산그래프가 세션 내에 생성된다.
model = tf.keras.Model(inputs=input, outputs=net, name="BASIC_MNIST")
model.summary() # 모델 요약

Model: "BASIC_MNIST"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                

# 최적화 계획 세우기
* `Loss Function`(손실 함수) - MSE, CEE를 쓸지를 결정
* `Optimization`(최적화 함수) - SGD, ADAM, RMSProps 등을 지정
  * 거의 대부분 ADAM, RMSProps 등을 기본으로 사용
* `Metrics` - 테스트 세트에 대한 평가 기준
  * `acc`, `recall`, `precision`

## Loss Function 선정 기준
* 이진 분류를 수행하는 경우(Binary Classification)
  * `mse` : 출력층의 뉴런이 1개인 경우 - with `sigmoid`
  * `cross entropy error` : 출력층의 뉴런이 2개인 경우 - with `softmax` ( 제일 자주 사용되는 방법 )
    * **`binary_crossentropy`**를 사용하면 된다.
* 다중 분류를 수행하는 경우(Multiclass Classification)
  * `cross entropy error` : 출력층의 뉴런을 클래스의 개수만큼 설정
    * **`categorical_crossentropy`**를 사용하면 된다.

## Categorical Cross Entropy의 종류
  * Label이 `[0, 1, 2]` 처럼 One-Hot Encoding이 되어있지 않은 경우
    * **`sparse_categorical_crossentropy`**를 사용
  * Label이 One-Hot Encoding이 되어있는 경우
    * **`categorical_crossentropy`**를 사용

**y_train**을 확인해서 어떤 Loss Function을 사용할지 결정해 보자

In [ ]:
y_train.shape

(60000,)

In [ ]:
y_train[:3]

array([5, 0, 4], dtype=uint8)

`y_train`확인 결과 OHE이 안되어 있다. 즉 `sparse_categorical_crossentropy`를 사용하거나, `y_train`을 OHE 한 다음 `categorical_crossentropy`를 사용하면 된다.

In [ ]:
loss_func = tf.keras.losses.sparse_categorical_crossentropy
loss_func

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy>

In [ ]:
# y_train이 OHE이 되어 있으면 categorical_crossentropy 사용
tf.keras.losses.categorical_crossentropy

<function tensorflow.python.keras.losses.categorical_crossentropy>

In [ ]:
# 이진 분류면? binary_crossentropy
tf.keras.losses.binary_crossentropy

<function tensorflow.python.keras.losses.binary_crossentropy>

## Optimizer 설정하기
* `sgd` - `tf.keras.optimizers.SGD()`
* `rmsprops` - `tf.keras.optimizers.RMSprop()`
* `adam` - `tf.keras.optimizers.Adam()` - 일반적으로 제일 많이 사용하는 최적화 기법, 뭘 쓸지 모르겠다면 `adam`을 선택하세요

In [ ]:
optm = tf.keras.optimizers.Adam()
optm

## 테스트 세트 평가방법(Metrics) 선정하기
  * 테스트 세트의 평가방식( evaluate )

In [ ]:
metrics = ['accuracy']
# metrics = ['acc']
# metrics = [tf.keras.metrics.Accuracy()]

# 모델 컴파일
* 텐서플로우 세션에 위치한 계산그래프에 데이터를 집어넣기 직전 작업을 완료
* 손실 함수, 최적화, 평가 방법들을 계산그래프에 적용

In [ ]:
model.compile(
    optimizer=optm,
    loss=loss_func,
    metrics=metrics
)

# 훈련용 하이퍼 파라미터 설정
  * 에폭 횟수(`num_epochs`)
  * 배치 크기(`batch_size`)

In [ ]:
num_epochs = 10
batch_size = 32

# 훈련 시작
* `fit`

In [ ]:
model.fit(
    X_train, # 훈련 X 데이터(feature)
    y_train, # 훈련 y 레이블(target)
    batch_size=batch_size, # 배치 사이즈
    epochs=num_epochs, # 에폭 횟수
    shuffle=True, # 데이터를 섞어가면서 훈련
    validation_split=0.2 # 검증 세트 비율
)

Epoch 1/10
1500/1500 [==============================] - 12s 7ms/step - loss: 0.2017 - accuracy: 0.9389 - val_loss: 0.1244 - val_accuracy: 0.9625
Epoch 2/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0846 - accuracy: 0.9734 - val_loss: 0.1039 - val_accuracy: 0.9696
Epoch 3/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0578 - accuracy: 0.9826 - val_loss: 0.0891 - val_accuracy: 0.9743
Epoch 4/10
1500/1500 [==============================] - 11s 8ms/step - loss: 0.0440 - accuracy: 0.9860 - val_loss: 0.1203 - val_accuracy: 0.9670
Epoch 5/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0349 - accuracy: 0.9885 - val_loss: 0.1064 - val_accuracy: 0.9725
Epoch 6/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0280 - accuracy: 0.9908 - val_loss: 0.1142 - val_accuracy: 0.9732
Epoch 7/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0232 - accuracy: 0.9932 - val_loss: 0.1012 - val_ac

# 런칭 직전 평가하는 시간
* `evaluate`

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.1181 - accuracy: 0.9746


[0.1181129738688469, 0.9746000170707703]